####CAPSTONE PROJECT

In [0]:
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.getOrCreate()

In [0]:
%sql
create catalog if not exists lakehouse

In [0]:
%sql
create schema if not exists lakehouse.lakehouse_schema

In [0]:
%sql
create volume if not exists lakehouse.lakehouse_schema.lakehouse_volume

####REFERENCE:
take()-Purpose: Fetch data into the driver program<br>
show()-Purpose: Display data in a readable table format (mainly for debugging/inspection)<br>
display()-Purpose: Rich, interactive visualization of data (Databricks-only)<br>
sample()-

syntam: df.sample(withReplacement=False, fraction=0.1, seed=42)
true-->can repeat ,false-->will not repeat
seed-->make same randomness for others if they use the same seed number

In [0]:
df=spark.read.csv("/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",header=False,inferSchema=True).toDF("id","fname","lname","age","profession")
#df.show(20,True)
display(df.take(20))
display(df.sample(.1))

In [0]:
df.printSchema()
print(df.columns)
print(df.dtypes)
print(df.schema) #- to identify the data using structtype and structfield format

In [0]:
#Important passive EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - 
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print(df.distinct().count())#row level 
display(df.distinct())#row level if id is same and name is different in 2 row means it will be treated as distinct only
print(df.dropDuplicates(["id"]).count()) #column level
display(df.dropDuplicates(["id"]))
print(df.describe())
print(df.summary())

In [0]:
#structuring of data -combining data + schema
struct1="id string,firstname string,lastname string,age string,profession string"
rawdf=spark.read.schema(struct1).csv("/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",header=False,inferSchema=True)
print(rawdf.schema)
#multiple lines with same or different names
df=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified","/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified"])
#multiple files in mulitple path
df=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified","/Volumes/telecom/telecom_schema/telecom_volume/custsmodified"],recursiveFileLookup=True,pathGlobalFilter="cust_*")

#pattern of files is mentioned

### structuring data

In [0]:
struct1 = "id string,firstname string,lastname string,age string,profession string"

rawdf1 = spark.read.schema(struct1).csv(
    "/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",
    header=False
)
rawdf2 = spark.read.schema(struct1).csv(
    "/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",
    header=False,
    mode="dropMalformed"
)

rawdf3 = rawdf1.union(rawdf2)
#)#Use union only if the dataframes are having same columns in the same order with same datatype
struct3 = "id string,name string,age string,city string,plan string"
rawdf4 = spark.read.schema(struct3).orc(
    "/Volumes/telecom_catalog_assign/landing_zone/ingestion_volume/cus_orc_out/"
).toDF("id", "name", "age", "city", "plan")

rawdf5 = rawdf2.unionByName(
    rawdf4,
    allowMissingColumns=True
)

display(rawdf5)
#files should have same data columns or even new columns can be added but here these are 2 different fileswith nolink so showing null

In [0]:
####2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,ShortType
stype=StructType([
    StructField("id",IntegerType(),True),
    StructField("firstname",StringType(),True),
    StructField("lastname",StringType(),True),
    StructField("age",ShortType(),True),
    StructField("profession",StringType(),True)
])
cleandf=spark.read.schema(stype).csv("/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",mode="permissive")
print("after keeping nulls on the wrong data format",cleandf.count())#all rows count
display(cleandf)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato) or
cleandf=spark.read.schema(stype).csv("/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",mode="dropMalformed")
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf.collect()))
display(cleandf)



In [0]:
#collect() is an action in PySpark that:
#Triggers execution of all previous transformations
#Brings the entire DataFrame / RDD data from executors to the driver
#Returns the data as a Python list ,make sure the data is 100% small
“When collect() is called, Spark executes one task per partition on executors, processes the data there, serializes each partition’s result, sends it over the network, and finally materializes everything in the driver’s memory."

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ShortType

stype = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("age", ShortType(), True),
    StructField("profession", StringType(), True)
])

cleandf = spark.read.schema(stype).csv(
    "/Volumes/lakehouse/lakehouse_schema/lakehouse_volume/custsmodified",
    mode="permissive",
    columnNameOfCorruptRecord="corrupt_record"
)

print("after keeping nulls on the wrong data format", cleandf.count())
cleandf.printSchema()
display(cleandf)
#rejecteddf1 = cleandf.where("corrupt_record is not null")-->this filters only the rows with corrupt_record
#print("corrupted record dataset", rejecteddf1.count())
#display(rejecteddf1) error while running this becoz of no corruptes records so no column is created 
#retaineddf = cleandf.where("corrupt_record is null")-->this filters only the rows with corrupt_record
#print("corrupted record dataset", retaineddf.count())
#print("Overall rows in the source data is ",len(cleandf.collect()))
#print("Rejected rows in the source data is ",len(rejectdf.collect()))
#print("Clean rows in the source data is ",len(retaineddf1.collect()))

In [0]:
#cleaning or cleansing -removing or deleting the corrupted records
cleandf2=cleandf.na.drop(how='any')# drops the row if any one of the column value is null
cleandf2=cleandf.na.drop(how='all')#drops the row only if all the column values are null
cleandf2=cleandf.na.drop(how='any',subset=['id','firstname'])#drops the row if any one of the column value is null in the subset columns
cleandf2=cleandf.na.drop(how='all',subset=['id','firstname'])#drops the row only if all the column values are null in the subset columns


In [0]:
#scrubbing- making the existing data tidy
cleandfnew=cleandf2.na.fill("not provided",subset=['profession','age'])
display(cleandfnew)
f_r={"Pilot":"Captain"}
cleandfnew=cleandf2.na.replace(f_r,subset=['profession'])
display(cleandfnew)
#as the name replace explains it searches the word and then replaces it as per provided in the disctionary